In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [17]:
from tensorflow.keras.callbacks import EarlyStopping
#Thông số
IMG_SIZE = (224, 224)
BATCH_SIZE = 64
EPOCHS = 5
TRAIN_DIR = '/kaggle/input/petimages/split_dataset/train'
TEST_DIR = '/kaggle/input/petimages/split_dataset/test'

#Load MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#Không fine-tune MobileNetV2
base_model.trainable = False

#Xây model mới
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # tốt hơn Flatten với mô hình pretrained
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

#Compile
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Data generator cho train + validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2  # 20% dữ liệu train sẽ được dùng làm validation
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generator cho training
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

# Generator cho validation
val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Generator cho test (giữ nguyên)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

#Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

#Save model
model.save('cat_dog_mobilenetv2.h5')
print("Huấn luyện xong, đã lưu mô hình vào 'cat_dog_mobilenetv2.h5'")

Found 16000 images belonging to 2 classes.
Found 3998 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 71/250 ━━━━━━━━━━━━━━━━━━━━ 38s 212ms/step - accuracy: 0.5450 - loss: 0.7626

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 287ms/step - accuracy: 0.6805 - loss: 0.5993 - val_accuracy: 0.9545 - val_loss: 0.2078
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 66s 259ms/step - accuracy: 0.9458 - loss: 0.2019 - val_accuracy: 0.9677 - val_loss: 0.1262
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 66s 258ms/step - accuracy: 0.9633 - loss: 0.1319 - val_accuracy: 0.9727 - val_loss: 0.0967
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 68s 265ms/step - accuracy: 0.9664 - loss: 0.1074 - val_accuracy: 0.9737 - val_loss: 0.0836
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 67s 263ms/step - accuracy: 0.9669 - loss: 0.0971 - val_accuracy: 0.9752 - val_loss: 0.0751
Huấn luyện xong, đã lưu mô hình vào 'cat_dog_mobilenetv2.h5'


In [18]:
loss, accuracy = model.evaluate(test_generator)

79/79 ━━━━━━━━━━━━━━━━━━━━ 17s 214ms/step - accuracy: 0.9773 - loss: 0.0677


In [19]:
from sklearn.metrics import classification_report

# Dự đoán trên tập test
y_true = test_generator.classes  # Nhãn thực tế
y_pred = model.predict(test_generator)
y_pred = (y_pred > 0.5).astype(int)  # Chuyển về nhãn 0 hoặc 1

# In các chỉ số phân loại
print(classification_report(y_true, y_pred))

79/79 ━━━━━━━━━━━━━━━━━━━━ 22s 243ms/step
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      2500
           1       0.97      0.98      0.98      2500

    accuracy                           0.98      5000
   macro avg       0.98      0.98      0.98      5000
weighted avg       0.98      0.98      0.98      5000

